## Deliverable 2. Create a Customer Travel Destinations Map.
----
1. Filter the DataFrame using input statements for minimum and maximum temperature preferences.
2. Drop any empty rows on the new DataFrame. 
3. Find the nearest hotel to each city using each city’s latitude and longitude and the Google Nearby Search.  
4. Create a new DataFrame where the rows that do not have a hotel are dropped from the DataFrame created in Step 4. 
5. Add the cities to a marker layer map with a pop-up marker for each city that includes: 
    - Hotel name
    - City
    - Country
    - Current weather description and the maximum temperature
6. Export the DataFrame as a CSV file into the Vacation_Search folder as WeatherPy_vacation.csv.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mataura,NZ,-46.19,168.86,39.00,83,2,3.00,clear sky
1,1,Narsaq,GL,60.92,-46.05,44.60,75,75,2.24,shower rain
2,2,Aksu,CN,41.12,80.26,82.42,22,6,5.08,clear sky
3,3,Provideniya,RU,64.38,-173.30,40.84,68,0,0.69,clear sky
4,4,Bilibino,RU,68.05,166.44,54.82,53,100,4.29,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria.. 
min_temp = float(input("What is the minimum temperature you would like for your vacation? "))
max_temp = float(input("What is the maximum temperature you would like for your vacation? "))

What is the minimum temperature you would like for your vacation? 70
What is the maximum temperature you would like for your vacation? 90


In [4]:
# 3. Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)] 
                                    
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Aksu,CN,41.12,80.26,82.42,22,6,5.08,clear sky
7,7,Touros,BR,-5.20,-35.46,80.60,74,75,11.41,broken clouds
10,10,Morehead,US,37.27,-87.18,75.20,78,1,3.58,clear sky
11,11,Port Elizabeth,ZA,-33.92,25.57,84.20,24,0,11.41,clear sky
16,16,Northam,GB,51.03,-4.22,82.40,48,20,9.17,few clouds
18,18,Hyderabad,IN,17.38,78.47,87.01,66,40,4.70,haze
20,20,Port Alfred,ZA,-33.59,26.89,82.00,30,0,12.12,clear sky
22,22,Atuona,PF,-9.80,-139.03,79.72,74,3,15.68,clear sky
23,23,Kapaa,US,22.08,-159.32,73.40,94,75,5.82,heavy intensity rain
25,25,Nikolskoye,RU,59.70,30.79,88.00,39,0,11.18,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                394
City                   394
Country                392
Lat                    394
Lng                    394
Max Temp               394
Humidity               394
Cloudiness             394
Wind Speed             394
Current Description    394
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_travel_cities = preferred_cities_df.dropna()
clean_travel_cities.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Aksu,CN,41.12,80.26,82.42,22,6,5.08,clear sky
7,7,Touros,BR,-5.20,-35.46,80.60,74,75,11.41,broken clouds
10,10,Morehead,US,37.27,-87.18,75.20,78,1,3.58,clear sky
11,11,Port Elizabeth,ZA,-33.92,25.57,84.20,24,0,11.41,clear sky
16,16,Northam,GB,51.03,-4.22,82.40,48,20,9.17,few clouds


In [7]:
clean_travel_cities.count()

City_ID                392
City                   392
Country                392
Lat                    392
Lng                    392
Max Temp               392
Humidity               392
Cloudiness             392
Wind Speed             392
Current Description    392
dtype: int64

In [8]:
clean_travel_cities.columns

Index(['City_ID', 'City', 'Country', 'Lat', 'Lng', 'Max Temp', 'Humidity',
       'Cloudiness', 'Wind Speed', 'Current Description'],
      dtype='object')

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_travel_cities[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name".
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Aksu,CN,82.42,clear sky,41.12,80.26,
7,Touros,BR,80.60,broken clouds,-5.20,-35.46,
10,Morehead,US,75.20,clear sky,37.27,-87.18,
11,Port Elizabeth,ZA,84.20,clear sky,-33.92,25.57,
16,Northam,GB,82.40,few clouds,51.03,-4.22,
18,Hyderabad,IN,87.01,haze,17.38,78.47,
20,Port Alfred,ZA,82.00,clear sky,-33.59,26.89,
22,Atuona,PF,79.72,clear sky,-9.80,-139.03,
23,Kapaa,US,73.40,heavy intensity rain,22.08,-159.32,
25,Nikolskoye,RU,88.00,clear sky,59.70,30.79,


In [10]:
# 6a. Set parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6c. Get latitude and longitude from DataFrame.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [11]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Aksu,CN,82.42,clear sky,41.12,80.26,Pudong Holiday Hotel
7,Touros,BR,80.60,broken clouds,-5.20,-35.46,INN NEW HORIZON
10,Morehead,US,75.20,clear sky,37.27,-87.18,CCI Express Inn
11,Port Elizabeth,ZA,84.20,clear sky,-33.92,25.57,39 On Nile Guest House
16,Northam,GB,82.40,few clouds,51.03,-4.22,Durrant House Hotel
18,Hyderabad,IN,87.01,haze,17.38,78.47,"Taj Krishna, Hyderabad"
20,Port Alfred,ZA,82.00,clear sky,-33.59,26.89,The Halyards Hotel
22,Atuona,PF,79.72,clear sky,-9.80,-139.03,Villa Enata
23,Kapaa,US,73.40,heavy intensity rain,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
25,Nikolskoye,RU,88.00,clear sky,59.70,30.79,Tourist House - Sablino


In [12]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"]!= '']
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Aksu,CN,82.42,clear sky,41.12,80.26,Pudong Holiday Hotel
7,Touros,BR,80.60,broken clouds,-5.20,-35.46,INN NEW HORIZON
10,Morehead,US,75.20,clear sky,37.27,-87.18,CCI Express Inn
11,Port Elizabeth,ZA,84.20,clear sky,-33.92,25.57,39 On Nile Guest House
16,Northam,GB,82.40,few clouds,51.03,-4.22,Durrant House Hotel
18,Hyderabad,IN,87.01,haze,17.38,78.47,"Taj Krishna, Hyderabad"
20,Port Alfred,ZA,82.00,clear sky,-33.59,26.89,The Halyards Hotel
22,Atuona,PF,79.72,clear sky,-9.80,-139.03,Villa Enata
23,Kapaa,US,73.40,heavy intensity rain,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
25,Nikolskoye,RU,88.00,clear sky,59.70,30.79,Tourist House - Sablino


In [13]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

###  Add a pop-up markers to each city. 
---
#### In the pop-up for each city add: 
1. The city name
2. The country code
3. The weather description and maximum temperature for the city

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))